In [2]:
from google.colab import drive
drive.mount('/content/drive/')

!cp "/content/drive/My Drive/Dissertation/envs/point_fall.py" .

Mounted at /content/drive/


In [17]:
# for inference, not continued training
def save_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/point_fall/{name}" 

    torch.save({
      'controller': {
          'critic': model.critic.state_dict(),
          'actor': model.actor.state_dict(),
      }
    }, path)

import copy
def load_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/point_fall/{name}" 
    checkpoint = torch.load(path)

    model.critic.load_state_dict(checkpoint['controller']['critic'])
    model.critic_target = copy.deepcopy(model.critic)
    
    model.actor.load_state_dict(checkpoint['controller']['actor'])
    model.actor_target = copy.deepcopy(model.actor)

    model.eval()

In [4]:
%matplotlib inline

import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from IPython import display
plt.ion()

# if gpu is to be used
device = torch.device("cuda")

In [5]:
class NormalizedEnv(gym.ActionWrapper):
    """ Wrap action """

    def action(self, action):
        act_k = (self.action_space.high - self.action_space.low)/ 2.
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k * action + act_b

    def reverse_action(self, action):
        act_k_inv = 2./(self.action_space.high - self.action_space.low)
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k_inv * (action - act_b)

In [6]:
from point_fall import PointFallEnv 
env = NormalizedEnv(PointFallEnv(4))

***

In [7]:
def plot_durations(episode_durations, actions):
    fig, axs = plt.subplots(2, figsize=(10,10))
    
    durations_t, durations = list(map(list, zip(*episode_durations)))
    durations = torch.tensor(durations, dtype=torch.float)
    
    fig.suptitle('Training')
    axs[0].set_xlabel('Episode')
    axs[0].set_ylabel('Reward')
    
    axs[0].plot(durations_t, durations.numpy())

    durations_t, durations = list(map(list, zip(*actions)))
    durations = torch.tensor(durations, dtype=torch.float)
    axs[1].plot(durations_t, durations.numpy())
        
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

In [8]:
# [reference] https://github.com/matthiasplappert/keras-rl/blob/master/rl/random.py

class RandomProcess(object):
    def reset_states(self):
        pass

class AnnealedGaussianProcess(RandomProcess):
    def __init__(self, mu, sigma, sigma_min, n_steps_annealing):
        self.mu = mu
        self.sigma = sigma
        self.n_steps = 0

        if sigma_min is not None:
            self.m = -float(sigma - sigma_min) / float(n_steps_annealing)
            self.c = sigma
            self.sigma_min = sigma_min
        else:
            self.m = 0.
            self.c = sigma
            self.sigma_min = sigma

    @property
    def current_sigma(self):
        sigma = max(self.sigma_min, self.m * float(self.n_steps) + self.c)
        return sigma


# Based on http://math.stackexchange.com/questions/1287634/implementing-ornstein-uhlenbeck-in-matlab
class OrnsteinUhlenbeckProcess(AnnealedGaussianProcess):
    def __init__(self, theta, mu=0., sigma=1., dt=1e-2, x0=None, size=1, sigma_min=None, n_steps_annealing=1000):
        super(OrnsteinUhlenbeckProcess, self).__init__(mu=mu, sigma=sigma, sigma_min=sigma_min, n_steps_annealing=n_steps_annealing)
        self.theta = theta
        self.mu = mu
        self.dt = dt
        self.x0 = x0
        self.size = size
        self.reset_states()

    def sample(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.current_sigma * np.sqrt(self.dt) * np.random.normal(size=self.size)
        self.x_prev = x
        self.n_steps += 1
        return x

    def reset_states(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros(self.size)

In [9]:
def soft_update(target, source, tau):
    for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - tau) + param.data * tau
        )

def hard_update(target, source):
    for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_(param.data)

In [10]:
# (state, action) -> (next_state, reward, done)
transition = namedtuple('transition', ('state', 'action', 'next_state', 'reward', 'done'))

# replay memory D with capacity N
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    # implemented as a cyclical queue
    def store(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        
        self.memory[self.position] = transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

***

In [11]:
class Actor(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(nb_states, 128)
        self.fc2 = nn.Linear(128, 128)
        self.head = nn.Linear(128, nb_actions)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return torch.tanh(self.head(x))

class Critic(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(Critic, self).__init__()

        # Q1 architecture
        self.l1 = nn.Linear(nb_states + nb_actions, 128)
        self.l2 = nn.Linear(128, 128)
        self.l3 = nn.Linear(128, 1)

        # Q2 architecture
        self.l4 = nn.Linear(nb_states + nb_actions, 128)
        self.l5 = nn.Linear(128, 128)
        self.l6 = nn.Linear(128, 1)
    
    def forward(self, state, action):
        sa = torch.cat([state, action], 1).float()

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)

        q2 = F.relu(self.l4(sa))
        q2 = F.relu(self.l5(q2))
        q2 = self.l6(q2)
        return q1, q2

    def Q1(self, state, action):
        sa = torch.cat([state, action], 1).float()

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)
        return q1

In [12]:
BATCH_SIZE = 64
GAMMA = 0.99

# https://spinningup.openai.com/en/latest/algorithms/td3.html
class TD3(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(TD3, self).__init__()
        self.nb_states = nb_states
        self.nb_actions= nb_actions
        
        self.actor = Actor(self.nb_states, self.nb_actions)
        self.actor_target = Actor(self.nb_states, self.nb_actions)
        self.actor_optimizer  = optim.Adam(self.actor.parameters(), lr=0.0001)

        self.critic = Critic(self.nb_states, self.nb_actions)
        self.critic_target = Critic(self.nb_states, self.nb_actions)
        self.critic_optimizer  = optim.Adam(self.critic.parameters(), lr=0.0001)

        hard_update(self.actor_target, self.actor)
        hard_update(self.critic_target, self.critic)
        
        #Create replay buffer
        self.memory = ReplayMemory(100000)
        self.random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=0.15, mu=0.0, sigma=0.2)

        # Hyper-parameters
        self.tau = 0.005
        self.depsilon = 1.0 / 10000
        self.policy_noise=0.2
        self.noise_clip=0.5
        self.policy_freq=2
        self.total_it = 0

        # 
        self.epsilon = 1.0
        self.is_training = True

    def update_policy(self):
        if len(self.memory) < BATCH_SIZE:
            return

        self.total_it += 1
        
        # in the form (state, action) -> (next_state, reward, done)
        transitions = self.memory.sample(BATCH_SIZE)
        batch = transition(*zip(*transitions))
        
        state_batch = torch.cat(batch.state)
        next_state_batch = torch.cat(batch.next_state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        done_mask = np.array(batch.done)
        not_done_mask = torch.from_numpy(1 - done_mask).float().to(device)

        # Target Policy Smoothing
        with torch.no_grad():
            # Select action according to policy and add clipped noise
            noise = (
                torch.randn_like(action_batch) * self.policy_noise
            ).clamp(-self.noise_clip, self.noise_clip).float()
            
            next_action = (
                self.actor_target(next_state_batch) + noise
            ).clamp(-1.0, 1.0).float()

            # Compute the target Q value
            # Clipped Double-Q Learning
            target_Q1, target_Q2 = self.critic_target(next_state_batch, next_action)
            target_Q = torch.min(target_Q1, target_Q2).squeeze(1)
            target_Q = (reward_batch + GAMMA * not_done_mask  * target_Q).float()
        
        # Critic update
        current_Q1, current_Q2 = self.critic(state_batch, action_batch)
      
        critic_loss = F.mse_loss(current_Q1, target_Q.unsqueeze(1)) + F.mse_loss(current_Q2, target_Q.unsqueeze(1))

        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Delayed policy updates
        if self.total_it % self.policy_freq == 0:
            # Compute actor loss
            actor_loss = -self.critic.Q1(state_batch, self.actor(state_batch)).mean()
            
            # Optimize the actor 
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            # Target update
            soft_update(self.actor_target, self.actor, self.tau)
            soft_update(self.critic_target, self.critic, self.tau)

    def eval(self):
        self.actor.eval()
        self.actor_target.eval()
        self.critic.eval()
        self.critic_target.eval()

    def observe(self, s_t, a_t, s_t1, r_t, done):
        self.memory.store(s_t, a_t, s_t1, r_t, done)

    def random_action(self):
        return torch.tensor([np.random.uniform(-1.,1.,self.nb_actions)], device=device, dtype=torch.float)

    def select_action(self, s_t, warmup, decay_epsilon):
        if warmup:
            return self.random_action()

        with torch.no_grad():
            action = self.actor(s_t).squeeze(0)
            #action += torch.from_numpy(self.is_training * max(self.epsilon, 0) * self.random_process.sample()).to(device).float()
            action += torch.from_numpy(self.is_training * max(self.epsilon, 0) * np.random.uniform(-1.,1.,1)).to(device).float()
            action = torch.clamp(action, -1., 1.)

            action = action.unsqueeze(0)
            
            if decay_epsilon:
                self.epsilon -= self.depsilon
            
            return action

In [13]:
import time
SAVE_OFFSET = 6

def train_model():
    global SAVE_OFFSET

    n_observations = env.observation_space.shape[0]
    n_actions = env.action_space.shape[0]
    
    agent = TD3(n_observations, n_actions).to(device)
    
    max_episode_length = 500
    
    agent.is_training = True
    episode_reward = 0.
    observation = None
    
    warmup = 200
    num_episodes = 4000 # M
    episode_durations = []
    actions = [(0,0)]

    steps = 0
    for i_episode in range(num_episodes):
        observation = env.reset()
        state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        
        overall_reward = 0
        episode_steps = 0
        done = False
        while not done:        
            # agent pick action ...
            action = agent.select_action(state, i_episode <= warmup, True)

            # env response with next_observation, reward, terminate_info
            action_i = action.detach().cpu().squeeze(0).numpy()

            observation, reward, done, info = env.step(action_i)
            steps += 1

            #actions.append((steps, action_i[0]))
                
            if max_episode_length and episode_steps >= max_episode_length - 1:
                done = True
            episode_steps += 1 
                
            extrinsic_reward = torch.tensor([reward], device=device)
            
            overall_reward += reward
            
            next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            # agent observe and update policy
            agent.observe(state, action, next_state, extrinsic_reward, done)           
            state = next_state
            
            if i_episode > warmup:
                agent.update_policy()

        episode_durations.append((i_episode, overall_reward))
        #plot_durations(episode_durations, actions)

        _, dur = list(map(list, zip(*episode_durations)))
        if len(dur) > 100:
            if i_episode % 100 == 0:
                print(f"{i_episode}: {np.mean(dur[-100:])}")
            if i_episode >= 400 and i_episode % 100 == 0 and np.mean(dur[-100:]) <= -49.0:
                print(f"Unlucky after {i_episode} eps! Terminating...")
                return None
            if np.mean(dur[-100:]) >= 90:
                print(f"Solved after {i_episode} episodes!")
                save_model(agent, f"td3_{SAVE_OFFSET}")
                SAVE_OFFSET += 1
                return agent
    return None

In [ ]:
i = 0
while i < 17:
    agent = train_model()
    
    if agent is not None:
        print(i)
        i += 1

In [ ]:
state_max = torch.from_numpy(env.observation_space.high).to(device).float()
state_min = torch.from_numpy(env.observation_space.low).to(device).float()
state_mid = (state_max + state_min) / 2.
state_range = (state_max - state_min)
def eval_model(agent, episode_durations):
    agent.eval()
    agent.is_training = False

    max_episode_length = 500
    num_episodes = 100

    for noise in np.arange(0,0.51,0.05):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()
            # unsqueeze adds batch dimension
            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:
                state = state + state_range * torch.FloatTensor(state.shape).uniform_(-noise, noise).to(device)
                state = torch.max(torch.min(state, state_max), state_min).float()

                action = agent.select_action(state, False, False)
                action_i = action.detach().cpu().squeeze(0).numpy()

                observation, reward, done, info = env.step(action_i) 
                
                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                
                episode_steps += 1 

                overall_reward += reward

                state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[np.round(noise, 2)].append(overall_reward / num_episodes)

In [14]:
state_max = torch.from_numpy(env.observation_space.high).to(device).float()
state_min = torch.from_numpy(env.observation_space.low).to(device).float()
state_mid = (state_max + state_min) / 2.
state_range = (state_max - state_min)
def fgsm_attack(data, eps, data_grad):
    sign_data_grad = data_grad.sign()

    perturbed_data = data - eps * sign_data_grad * state_range

    clipped_perturbed_data = torch.max(torch.min(perturbed_data, state_max), state_min)

    return clipped_perturbed_data

def fgsm_action(state, agent, eps, target, targeted):
    state = state.clone().detach().requires_grad_(True)

    if targeted:
        # initial forward pass
        action = agent.actor(state)
        action = torch.clamp(action, -1., 1.)

        loss = F.mse_loss(action, target)
    else:
        loss = agent.critic.Q1(state, agent.actor(state)).mean()

    agent.actor.zero_grad()
    agent.critic.zero_grad()

    # calc loss
    loss.backward()
    data_grad = state.grad.data
    # perturb state
    state_p = fgsm_attack(state, eps, data_grad).float()
    return agent.select_action(state_p, False, False)

def apply_fgsm(agent, episode_durations, targeted):
    TARGET_ACTION = torch.tensor([[0.0, 0.0]], device=device, dtype=torch.float)

    agent.eval()

    max_episode_length = 500
    agent.is_training = False

    num_episodes = 100

    for eps in np.arange(0.0, 0.201, 0.02):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()
            # unsqueeze adds batch dimension
            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:
                action = fgsm_action(state, agent, eps, TARGET_ACTION, targeted)
                action_i = action.detach().cpu().squeeze(0).numpy()

                observation, reward, done, info = env.step(action_i)
                
                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                
                episode_steps += 1 

                overall_reward += reward

                state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[eps].append(overall_reward / num_episodes)

In [ ]:
def plot_norms(episode_durations):
    plt.figure(2, figsize=(10,10))
    
    x, ys = np.array(list(episode_durations.keys())), np.array(list(episode_durations.values()))
    
    plt.title('Action Prediction $\mu$ and $\pm \sigma$ interval')
    plt.xlabel('L2 Norm')
    plt.ylabel('Average Reward')
    
    mu = np.mean(ys, axis=1)
    plt.plot(x, mu)
    stds = np.std(ys, axis = 1)
    plt.fill_between(x, mu + stds , mu - stds, alpha=0.2)
        
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

In [18]:
episodes = {}
for l2norm in np.arange(0,0.51,0.05):
    episodes[np.round(l2norm, 2)] = []

fgsm_t = {}
fgsm_ut = {}
for eps in np.arange(0.0, 0.201, 0.02):
    fgsm_t[eps] = []
    fgsm_ut[eps] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

"""
i = 0
while i < 11:
    #agent = train_model()
    agent = TD3(n_observations, n_actions).to(device)
    load_model(agent, f"td3_{i}")

    if agent is not None:
        eval_model(agent, episodes)
        print(f"{i} noise: {episodes}")
        i += 1

print("---")
print(f"noise: {episodes}")
"""

for i in [0, 1, 2, 3, 4]:
    #agent = train_model()
    agent = TD3(n_observations, n_actions).to(device)
    load_model(agent, f"td3_{i}")

    if agent is not None:
        apply_fgsm(agent, fgsm_t, True)
        apply_fgsm(agent, fgsm_ut, False)
        print(f"{i} fgsm (t): {fgsm_t}")
        print(f"{i} fgsm (ut): {fgsm_ut}")
        i += 1

print("---")
print(f"fgsm (t): {fgsm_t}")
print(f"fgsm (ut): {fgsm_ut}")

0 fgsm (t): {0.0: [97.80999999999597], 0.02: [-41.63899999997277], 0.04: [-17.889999999986298], 0.06: [-17.354999999987832], 0.08: [-35.579999999974106], 0.1: [-42.07299999997629], 0.12: [-44.86999999998728], 0.14: [-47.228999999994684], 0.16: [-48.62800000000068], 0.18: [-48.97400000000308], 0.2: [-48.68700000000204]}
0 fgsm (ut): {0.0: [97.56799999999632], 0.02: [54.48399999998769], 0.04: [-14.542999999999903], 0.06: [2.544999999997841], 0.08: [3.281999999994613], 0.1: [-20.574999999984144], 0.12: [-31.809999999971534], 0.14: [-38.66899999997551], 0.16: [-45.171999999987094], 0.18: [-50.00000000000659], 0.2: [-47.69399999999638]}
1 fgsm (t): {0.0: [97.80999999999597, 95.60599999999312], 0.02: [-41.63899999997277, -46.795999999996745], 0.04: [-17.889999999986298, -34.43199999997088], 0.06: [-17.354999999987832, -36.16599999996913], 0.08: [-35.579999999974106, 9.039999999998228], 0.1: [-42.07299999997629, -0.01799999999945533], 0.12: [-44.86999999998728, -22.303999999976803], 0.14: [-4

In [ ]:
print("noise: {0.0: [97.81499999999596, 95.79099999999157, 96.29499999999415, 97.93399999999622, 89.30499999998712, 97.84699999999604, 74.62099999997754, 95.85599999999204, 96.18399999999309, 97.01099999999464, 96.47899999999447], 0.05: [39.19600000001345, 11.098999999999204, 40.6430000000141, 76.87199999998931, -38.2709999999691, 22.212000000009603, -5.862000000002688, 48.65999999999616, 60.307999999991196, 51.18599999999777, 29.395000000016207], 0.1: [39.44700000001104, 19.49400000000553, 7.407999999998801, 5.172999999996604, -29.545999999980477, -0.7610000000016894, 15.351000000007309, 54.35499999999417, 44.139000000016075, 5.32299999999485, -11.879000000005778], 0.15: [34.88800000001661, 10.176000000005898, -4.807000000003346, 1.80099999999872, -37.58199999997086, -7.636999999988667, 10.978999999999944, 49.05499999999026, 19.37700000001689, 9.861999999997536, -25.572999999975647], 0.2: [15.112000000010319, 1.2659999999980365, -11.567999999999508, 1.0520000000000012, -38.732999999969586, -17.227999999990075, -2.1740000000002775, 35.37600000001522, 11.374000000009316, -15.644999999990025, -44.16399999998171], 0.25: [-3.8680000000034487, 11.804000000015783, -14.883999999984407, 9.46299999999308, -39.18599999997031, -23.24799999998533, -4.704000000004297, 48.1570000000026, 4.500999999992392, -4.468000000003668, -48.53400000000148], 0.3: [-31.383999999974453, -1.502000000002005, -28.093999999980177, 15.916000000008475, -43.60099999997876, -34.167999999971215, -0.6040000000000825, 25.675000000019203, -23.599999999983996, -16.84399999998561, -45.650999999992585], 0.35: [-39.02299999996841, 7.255999999997249, -14.482999999988003, -3.4680000000078066, -45.07599999999413, -22.656999999984663, -9.733999999996666, 29.077000000016533, -23.853999999984406, -16.208999999992002, -47.14599999999438], 0.4: [-25.800999999977197, 0.5730000000010412, -22.76899999998588, 3.8259999999933054, -44.070999999980465, -25.158999999975304, 6.500999999997405, 1.9169999999969856, -35.369999999973274, -20.191999999981523, -50.00000000000659], 0.45: [-34.308999999973565, 2.224999999999611, -11.166999999994822, -6.007000000003689, -41.651999999972986, -29.768999999973133, -2.7340000000000577, -0.8810000000035015, -26.87399999997508, -14.243999999991024, -45.655999999988914], 0.5: [-31.747999999974258, 0.0439999999931584, -17.907999999982874, -5.305000000004871, -44.36099999998351, -31.96599999997717, -13.580999999999328, 3.650999999998195, -33.08199999996927, -19.990999999988492, -47.15799999999897]}")
print("fgsm (t): {0.0: [95.57199999999243, 96.19799999999323, 94.81699999999242, 97.9369999999962, 96.64399999999395, 97.85999999999605, 77.05699999997755, 95.92999999999223, 96.2319999999931, 97.01499999999461, 97.96599999999627], 0.02: [35.51200000001988, 90.58899999998599, 73.89699999999209, 97.86099999999605, 22.522000000013207, -20.712999999990046, -26.80899999997856, 77.60999999998342, 80.30499999998672, 31.939000000019064, 89.75199999998918], 0.04: [26.968000000013845, 65.82299999997052, 1.3969999999971012, 97.82599999999601, -11.029000000002043, -30.563999999977636, -40.824999999974466, 46.20999999999979, -14.923999999991544, 15.293000000012748, 78.39199999999352], 0.06: [58.230999999987546, 9.583000000000402, -12.918999999990211, 92.02999999999116, -24.49799999997799, -27.016999999972967, -13.35099999999957, 55.218999999998566, -39.74599999996958, -9.693999999991156, 74.25999999997873], 0.08: [23.172000000012527, 24.187000000015917, -21.81999999998329, 67.29799999999072, -33.33399999997277, 16.935000000008753, -17.590999999976752, 68.79399999997432, -41.49399999997805, 29.113000000009084, 77.17699999999063], 0.1: [53.25399999999027, -0.2180000000023696, -12.198999999996294, -2.6900000000004285, -23.889999999981434, 5.224999999996488, -16.77999999998918, 49.12600000000882, -44.240999999981995, -15.395999999989506, 91.66699999998907], 0.12: [8.550999999994978, 4.264999999995929, -16.995999999992122, 2.5499999999963503, -31.487999999974814, 1.6220000000001022, -23.45399999997804, 7.224999999995686, -47.94599999999843, -33.16599999996982, 17.830000000009445], 0.14: [-30.374999999974666, -9.393999999993381, 15.905000000014029, 8.213999999998551, -29.69999999997737, 6.232999999995797, -17.91199999998686, -24.67299999997783, -47.961999999999215, -33.98299999996913, -48.90400000000169], 0.16: [-29.086999999976868, -17.957999999990374, 37.8360000000115, -2.723000000001943, -30.860999999973792, 3.4049999999957494, -20.460999999987468, -27.45899999997346, -45.63099999998819, -47.39000000000087, -50.00000000000659], 0.18: [-45.469999999988744, -26.167999999978083, 43.729000000003595, 5.205999999996762, -27.704999999978522, -11.222000000003717, -15.169999999980416, -29.884999999981346, -46.85599999999242, -44.8159999999875, -50.00000000000659], 0.2: [-40.65899999996761, -18.59899999998767, 8.997999999994933, -22.16799999998375, -22.78599999998103, -3.604999999999342, -25.204999999973253, -32.1539999999753, -43.005999999975685, -38.074999999970984, -50.00000000000659]}")

noise: {0.0: [97.81499999999596, 95.79099999999157, 96.29499999999415, 97.93399999999622, 89.30499999998712, 97.84699999999604, 74.62099999997754, 95.85599999999204, 96.18399999999309, 97.01099999999464, 96.47899999999447], 0.05: [39.19600000001345, 11.098999999999204, 40.6430000000141, 76.87199999998931, -38.2709999999691, 22.212000000009603, -5.862000000002688, 48.65999999999616, 60.307999999991196, 51.18599999999777, 29.395000000016207], 0.1: [39.44700000001104, 19.49400000000553, 7.407999999998801, 5.172999999996604, -29.545999999980477, -0.7610000000016894, 15.351000000007309, 54.35499999999417, 44.139000000016075, 5.32299999999485, -11.879000000005778], 0.15: [34.88800000001661, 10.176000000005898, -4.807000000003346, 1.80099999999872, -37.58199999997086, -7.636999999988667, 10.978999999999944, 49.05499999999026, 19.37700000001689, 9.861999999997536, -25.572999999975647], 0.2: [15.112000000010319, 1.2659999999980365, -11.567999999999508, 1.0520000000000012, -38.732999999969586, -

In [ ]:
def eval_scale(agent, episode_durations):
    agent.eval()
    agent.is_training = False

    max_episode_length = 500
    num_episodes = 100

    for scale in np.arange(1.0,7.01,0.5):
        env = NormalizedEnv(PointFallEnv(scale))

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()
            # unsqueeze adds batch dimension
            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:

                action = agent.select_action(state, False, False)
                action_i = action.detach().cpu().squeeze(0).numpy()

                observation, reward, done, info = env.step(action_i) 
                
                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                
                episode_steps += 1 

                overall_reward += reward

                state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[np.round(scale, 2)].append(overall_reward / num_episodes)

In [ ]:
episodes = {}
for scale in np.arange(1.0,7.01,0.5):
    episodes[np.round(scale, 2)] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

i = 0
while i < 11:
    agent = TD3(n_observations, n_actions).to(device)
    load_model(agent, f"td3_{i}")

    if agent is not None:
        eval_scale(agent, episodes)
        print(f"{i} scale: {episodes}")
        i += 1

print("---")
print(f"scale: {episodes}")

0 scale: {1.0: [-35.835999999973694], 1.5: [-35.9149999999692], 2.0: [-37.89399999997111], 2.5: [33.00800000001212], 3.0: [55.47699999998554], 3.5: [98.00299999999628], 4.0: [97.819999999996], 4.5: [97.67999999999573], 5.0: [17.665000000001974], 5.5: [57.4319999999789], 6.0: [74.99299999997442], 6.5: [1.3509999999953681], 7.0: [-38.95499999996939]}
1 scale: {1.0: [-35.835999999973694, 97.7049999999989], 1.5: [-35.9149999999692, -35.61599999997276], 2.0: [-37.89399999997111, -15.921999999994489], 2.5: [33.00800000001212, 69.35199999998638], 3.0: [55.47699999998554, 89.27799999999148], 3.5: [98.00299999999628, 94.66999999999138], 4.0: [97.819999999996, 96.22099999999323], 4.5: [97.67999999999573, 96.37099999999342], 5.0: [17.665000000001974, 96.03299999999261], 5.5: [57.4319999999789, 80.88099999998138], 6.0: [74.99299999997442, 73.57499999998119], 6.5: [1.3509999999953681, 45.81000000000228], 7.0: [-38.95499999996939, 9.903000000005465]}
2 scale: {1.0: [-35.835999999973694, 97.704999999

FileNotFoundError: ignored

In [ ]:
def eval_starting_position(agent, episode_durations):
    agent.eval()
    agent.is_training = False

    max_episode_length = 500
    num_episodes = 100

    for extra_range in np.arange(0.0, 0.401, 0.05):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()

            extra = np.random.uniform(-0.1 - extra_range, 0.1 + extra_range, env.starting_point.shape)
            #extra = np.random.uniform(0.1, 0.1 + extra_range, env.starting_point.shape)
            #extra = extra * (2*np.random.randint(0,2,size=env.starting_point.shape)-1)
            env.unwrapped.state = np.array(env.starting_point + extra, dtype=np.float32)
            env.unwrapped.state[2] += math.pi / 2. # start facing up
            env.unwrapped.state[2] = env.state[2] % (2 * math.pi)
            observation = env.normalised_state()

            # unsqueeze adds batch dimension
            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:

                action = agent.select_action(state, False, False)
                action_i = action.detach().cpu().squeeze(0).numpy()

                observation, reward, done, info = env.step(action_i) 
                
                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                
                episode_steps += 1 

                overall_reward += reward

                state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[np.round(extra_range, 3)].append(overall_reward / num_episodes)

In [ ]:
episodes = {}
for extra_range in np.arange(0.0, 0.401, 0.05):
    episodes[np.round(extra_range, 3)] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

i = 0
while i < 11:
    agent = TD3(n_observations, n_actions).to(device)
    load_model(agent, f"td3_{i}")

    if agent is not None:
        eval_starting_position(agent, episodes)
        print(f"{i} extra_range: {episodes}")
        i += 1

print("---")
print(f"extra_range: {episodes}")

0 extra_range: {0.0: [96.13399999999586], 0.05: [97.84299999999601], 0.1: [96.36099999999662], 0.15: [88.99399999998981], 0.2: [79.92499999998046], 0.25: [76.99499999998858], 0.3: [69.98099999998173], 0.35: [63.36599999999035], 0.4: [58.85999999999196]}
1 extra_range: {0.0: [96.13399999999586, 96.26499999999332], 0.05: [97.84299999999601, 93.20899999999355], 0.1: [96.36099999999662, 69.93699999998559], 0.15: [88.99399999998981, 58.15099999998757], 0.2: [79.92499999998046, 51.99899999999083], 0.25: [76.99499999998858, 34.228000000016856], 0.3: [69.98099999998173, 47.098000000009854], 0.35: [63.36599999999035, 19.86500000000292], 0.4: [58.85999999999196, 10.691000000010465]}
2 extra_range: {0.0: [96.13399999999586, 96.26499999999332, 96.30299999999419], 0.05: [97.84299999999601, 93.20899999999355, 84.45099999999358], 0.1: [96.36099999999662, 69.93699999998559, 71.15099999999032], 0.15: [88.99399999998981, 58.15099999998757, 57.8159999999806], 0.2: [79.92499999998046, 51.99899999999083, 4

In [ ]:
state_max = torch.from_numpy(env.observation_space.high).to(device).float()
state_min = torch.from_numpy(env.observation_space.low).to(device).float()
state_mid = (state_max + state_min) / 2.
state_range = (state_max - state_min)
def save_trajectories(agent, episode_durations, dirty):
    agent.eval()

    max_episode_length = 500
    agent.is_training = False

    num_episodes = 10

    l2norm = 0.3
    episode_durations.append([])
    
    for i_episode in range(num_episodes):
        path = {"overall_reward": 0, "worker": []}

        observation = env.reset()

        state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        state_ = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_steps = 0
        overall_reward = 0
        done = False
        while not done:
            if dirty:
                state = state + state_range * torch.FloatTensor(state.shape).uniform_(-l2norm, l2norm).to(device)
                state = torch.max(torch.min(state, state_max), state_min).float()

            action = agent.select_action(state, False, False)
            action_i = action.detach().cpu().squeeze(0).numpy()
            path["worker"].append((episode_steps, state_.detach().cpu().squeeze(0).numpy(), action.detach().cpu().squeeze(0).numpy()))

            observation, reward, done, info = env.step(action_i) 
            
            if max_episode_length and episode_steps >= max_episode_length - 1:
                done = True
            
            episode_steps += 1 

            overall_reward += reward

            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
            state_ = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        path["overall_reward"] = overall_reward
        episode_durations[-1].append(path)

In [ ]:
episodes = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

i = 0
while i < 11:
    #agent = train_model()
    agent = TD3(n_observations, n_actions).to(device)
    load_model(agent, f"td3_{i}")

    if agent is not None:
        # goal_attack, action_attack, same_noise
        save_trajectories(agent, episodes, True)
        #print(f"{i} paths: {episodes}")
        i += 1

print("----")
#print(f"paths: {episodes}")

episodes.pop(6)

torch.save(episodes, "td3_PointFall_dirty_eps.pt")

----
